In [3]:
import boto3
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

In [ ]:
session = sagemaker.Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and session is not None:
    sagemaker_session_bucket = session.default_bucket()

In [ ]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

session = sagemaker.Session(default_bucket=sagemaker_session_bucket)

In [ ]:
print(f"Sagemaker Role ARN: {role}")
print(f"Sagemaker Session Region: {session.boto_region_name}")

sagemaker role arn: arn:aws:iam::136566696263:role/service-role/AmazonSageMaker-ExecutionRole-20240513T112438
sagemaker session region: us-east-1


In [ ]:
hub = {
  'HF_MODEL_ID': 'distilbert-base-uncased-distilled-squad',
  'HF_TASK': 'question-answering'                           
}

In [ ]:
huggingface_model = HuggingFaceModel(
   env=hub,                                               
   role=role,                                            
   transformers_version="4.26",                            
   pytorch_version="1.13",                                
   py_version='py39',                                    
)

In [ ]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

In [ ]:
data = {
    "inputs": {
        "question": "What tool is used for inference?",
        "context": "My name is Andrew, and I live in Cairo. This model is used with SageMaker for inference."
    }
}

In [ ]:
predictor.predict(data)

-----!

{'score': 0.9987204670906067, 'start': 68, 'end': 77, 'answer': 'sagemaker'}

In [ ]:
data = {
    "inputs": {
        "question": "What does Andrew teach?",
        "context": "My name is Andrew and I teach data science."
    }
}

In [13]:
predictor.predict(data)

{'score': 0.9854100942611694, 'start': 30, 'end': 42, 'answer': 'data science'}